http://docs.sqlalchemy.org/en/latest/_modules/examples/performance/bulk_inserts.html

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Reload all modules imported with %aimport
%load_ext autoreload
%autoreload 1

In [2]:
import pandas as pd
idx = pd.IndexSlice

In [3]:
from odo import *

In [4]:
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, create_engine, bindparam
from sqlalchemy.orm import Session
from sqlalchemy.exc import SQLAlchemyError

Base = declarative_base()

In [5]:
dburl = 'sqlite:////tmp/foo.db'

In [6]:
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Date, Float,create_engine, bindparam
from sqlalchemy.orm import Session


In [7]:
Base = declarative_base()

class Price(Base):
    __tablename__ = "prices"
    Ticker = Column(String(255), primary_key=True)
    Date   = Column(Date, primary_key=True)
    AdjClose = Column(Float)
    Close   = Column(Float)
    High    = Column(Float)
    Low     = Column(Float)
    Open    = Column(Float)
    Volume  = Column(Float)
    

In [8]:
def setup_database(dburl, echo, num):
    engine = create_engine(dburl, echo=echo)
    Base.metadata.drop_all(engine)
    Base.metadata.create_all(engine)
    
    return engine


In [9]:
engine = setup_database(dburl, True, 0)

2018-04-05 12:46:02,896 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2018-04-05 12:46:02,897 INFO sqlalchemy.engine.base.Engine ()
2018-04-05 12:46:02,899 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2018-04-05 12:46:02,900 INFO sqlalchemy.engine.base.Engine ()
2018-04-05 12:46:02,901 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("prices")
2018-04-05 12:46:02,902 INFO sqlalchemy.engine.base.Engine ()
2018-04-05 12:46:02,904 INFO sqlalchemy.engine.base.Engine 
DROP TABLE prices
2018-04-05 12:46:02,905 INFO sqlalchemy.engine.base.Engine ()
2018-04-05 12:46:02,916 INFO sqlalchemy.engine.base.Engine COMMIT
2018-04-05 12:46:02,917 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("prices")
2018-04-05 12:46:02,917 INFO sqlalchemy.engine.base.Engine ()
2018-04-05 12:46:02,919 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE prices (
	"Ticker" VARCHAR(255) NOT NULL, 
	"Date" DATE NOT N

In [10]:
dfs = []

for t in ("a", "b"):
    df = pd.read_csv("/tmp/" + t + ".csv")
    df["Ticker"] = t
    df = df.rename( columns={ "Adj Close": "AdjClose"} )
    
    df["Date"] = pd.to_datetime( df["Date"])
    
    dfs.append(df)              
                         
df_big_thin = pd.concat( dfs )

In [11]:
df_big_thin.shape

(2883, 8)

In [12]:
from sqlalchemy.sql import and_, or_, not_, func




In [13]:
session_add = Session(bind=engine)
rows = df_big_thin.to_dict("records")

for row in rows:
    rec = Price(**row)
    
    session_add.add(rec)

try:
    session_add.flush()
except SQLAlchemyError as e:
    print("Flush error: {}".format(e))
    
session_add.commit()    

2018-04-05 12:46:03,236 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-04-05 12:46:03,329 INFO sqlalchemy.engine.base.Engine INSERT INTO prices ("Ticker", "Date", "AdjClose", "Close", "High", "Low", "Open", "Volume") VALUES (?, ?, ?, ?, ?, ?, ?, ?)
2018-04-05 12:46:03,331 INFO sqlalchemy.engine.base.Engine (('a', '2013-02-01', 29.858995, 31.01, 31.74, 30.469998999999998, 31.5, 66789100.0), ('a', '2013-02-04', 29.868622, 31.02, 31.99, 30.76, 31.09, 7695400.0), ('a', '2013-02-05', 29.887882, 31.040001, 31.98, 30.85, 31.25, 5013200.0), ('a', '2013-02-06', 29.878252000000003, 31.030001000000002, 31.43, 30.75, 30.98, 2126100.0), ('a', '2013-02-07', 30.812247999999997, 32.0, 32.73, 31.0, 31.0, 3800800.0), ('a', '2013-02-08', 31.823278000000002, 33.049999, 33.48, 32.299999, 32.310001, 2599300.0), ('a', '2013-02-11', 32.025482000000004, 33.259997999999996, 33.5, 32.880001, 33.060001, 1487200.0), ('a', '2013-02-12', 32.487659, 33.740002000000004, 34.0, 33.150002, 33.400002, 1637300.0)

In [14]:
rsession = Session(bind=engine)

## Find max date per ticker

In [15]:
# q = session_add.query(Price).filter( and_(Price.Ticker=="a", Price.Date>="2018-03-27") )
q = rsession.query(Price, Price.Ticker, func.max(Price.Date).label("max_date")).group_by(Price.Ticker)
l = q.all()
maxDate = {}
for a in l:
    maxDate[a.Ticker] = a.max_date
    print(a.Ticker, a.max_date)

maxDate

2018-04-05 12:46:03,527 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-04-05 12:46:03,531 INFO sqlalchemy.engine.base.Engine SELECT prices."Ticker" AS "prices_Ticker", prices."Date" AS "prices_Date", prices."AdjClose" AS "prices_AdjClose", prices."Close" AS "prices_Close", prices."High" AS "prices_High", prices."Low" AS "prices_Low", prices."Open" AS "prices_Open", prices."Volume" AS "prices_Volume", max(prices."Date") AS max_date 
FROM prices GROUP BY prices."Ticker"
2018-04-05 12:46:03,532 INFO sqlalchemy.engine.base.Engine ()
a 2018-04-03
b 2018-04-03


{'a': datetime.date(2018, 4, 3), 'b': datetime.date(2018, 4, 3)}

In [16]:
ticker_up = "a"
df_up = pd.read_csv("/tmp/" + ticker_up + "_up.csv")

FileNotFoundError: File b'/tmp/a_up.csv' does not exist

In [ ]:
dates_up = df_up["Date"].tolist()
dates_up

In [17]:
q = rsession.query(Price).filter( and_(Price.Ticker == ticker_up, Price.Date.in_(dates_up)) )
l = q.all()
for a in l:
    print(a.Ticker, a.Date)


NameError: name 'dates_up' is not defined

## Delete rows

In [ ]:
q  = rsession.query(Price).filter(and_(Price.Ticker == ticker_up, Price.Date.in_(dates_up)))
print(q.all())
num_rows = q.delete(synchronize_session="fetch") # or = False
print("Deleted {} rows".format(num_rows))

In [ ]:
q = rsession.query(Price)
resps = q.all()
for r in resps[0:10]:
    print(r.Ticker, r.Date)


In [ ]:
type(q)
l = q.all()
len(l)
type(l)
l[0]

## Read from db; form wide df

In [ ]:
df_r = pd.read_sql(rsession.query(Price).statement, rsession.bind)
df_r[ "Dt" ] = df_r.loc[:,"Date"].map( lambda  s: pd.to_datetime(s, infer_datetime_format=True) )
df_r = df_r.drop("Date", axis=1)
df_r.set_index(["Dt", "Ticker"], inplace=True)
df_w = df_r.unstack(level=1)
df_w.tail()

df_w.loc["2018-03-29", idx[ "AdjClose",:]]

## Convert wide df to think table to store in db

In [ ]:
df_t = df_w.stack(level=1)
df_t.reset_index(inplace=True)
df_t.tail()
df_t.columns

In [ ]:
def test_flush_no_pk(n):
    """Individual INSERT statements via the ORM, calling upon last row id"""
    session = Session(bind=engine)
    for chunk in range(0, n, 1000):
        session.add_all([
            Customer(
                name='customer name %d' % i,
                description='customer description %d' % i)
            for i in range(chunk, chunk + 1000)
        ])
        session.flush()
    session.commit()


In [ ]:
fname = "/tmp/a.csv"
ds = discover( resource(fname) )
ds